In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from secret import *
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(
    url = "bolt://localhost:7687",
    username = "neo4j",
    password = "test1234"
)

In [3]:
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)

[]

In [4]:
graph.refresh_schema()

In [5]:
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [6]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer

diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=DIFFBOT_API_TOKEN)

In [7]:
from langchain.document_loaders import WikipediaLoader

query = "pulmonary edema"
# query = "Atelectasis"
# query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()
graph_documents, all_results = diffbot_nlp.convert_to_graph_documents(raw_documents)
# took 1m4.0s

In [8]:
all_results

[{'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []},
 {'facts': []}]

In [8]:
graph_documents

[GraphDocument(nodes=[], relationships=[], source=Document(page_content='Pulmonary edema, also known as pulmonary congestion, is excessive liquid accumulation in the tissue and air spaces (usually alveoli) of the lungs. It leads to impaired gas exchange and may cause hypoxemia and respiratory failure. It is due to either failure of the left ventricle of the heart to remove oxygenated blood adequately from the pulmonary circulation (cardiogenic pulmonary edema), or an injury to the lung tissue directly or blood vessels of the lung (non-cardiogenic pulmonary edema).Treatment is focused on three aspects: firstly improving respiratory function, secondly, treating the underlying cause, and thirdly preventing further damage and assuring full recovery to the lung. Pulmonary edema, especially when sudden (acute), can lead to respiratory failure or cardiac arrest due to hypoxia. It is a cardinal feature of congestive heart failure. The term edema is from the Greek οἴδημα (oidēma, "swelling"), f

In [8]:
graph.add_graph_documents(graph_documents)

In [9]:
graph.refresh_schema()

In [10]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Organization', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}, {'property': 'foundingDate', 'type': 'STRING'}]}, {'labels': 'Person', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'positionHeld', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'age', 'type': 'STRING'}, {'property': 'dateOfBirth', 'type': 'STRING'}, {'property': 'numberOfChildren', 'type': 'STRING'}, {'property': 'academicDegree', 'type': 'STRING'}, {'property': 'dateOfDeath', 'type': 'STRING'}]}, {'labels': 'Skill', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Location', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Disease', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Award', 'propert

In [11]:
chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k"),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k"),
    graph=graph,
    verbose=True,
)

In [12]:
# chain.run("Which university did Warren Buffett attend?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:EDUCATED_AT]->(o:Organization)
WHERE p.name = 'Warren Buffett'
RETURN o.name
Full Context:
[{'o.name': 'University of Nebraska-Lincoln'}, {'o.name': 'New York Institute of Finance'}, {'o.name': "Penn's Fels Institute of Government"}, {'o.name': 'Woodrow Wilson High School'}, {'o.name': 'Columbia Business School'}, {'o.name': 'Alice Deal Junior High School'}]

> Finished chain.


'Warren Buffett attended the University of Nebraska-Lincoln.'

In [13]:
# chain.run("What's Atelectasis?")

In [14]:
# print(graph.schema)


        Node properties are the following:
        [{'labels': 'Organization', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}, {'property': 'foundingDate', 'type': 'STRING'}]}, {'labels': 'Person', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'positionHeld', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'age', 'type': 'STRING'}, {'property': 'dateOfBirth', 'type': 'STRING'}, {'property': 'numberOfChildren', 'type': 'STRING'}, {'property': 'academicDegree', 'type': 'STRING'}, {'property': 'dateOfDeath', 'type': 'STRING'}]}, {'labels': 'Skill', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Location', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Disease', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Award', 'propert